In [1]:
import pandas as pd
import numpy as np
import slugify
import datetime as dt
import json
from IPython.display import Markdown as md
%load_ext watermark

def save_json(filename, data):
    with open(f"resources/{filename}", "w") as afile:
        json.dump(data, afile)

def open_json(filename):
    with open(f"resources/{filename}", "r") as afile:
        data = json.load(afile)
        return data

# this defines the css rules for the note-book table displays
header_row = {'selector': 'th:nth-child(1)', 'props': f'background-color: #FFF; text-align:right'}
even_rows = {"selector": 'tr:nth-child(even)', 'props': f'background-color: rgba(139, 69, 19, 0.08);'}
odd_rows = {'selector': 'tr:nth-child(odd)', 'props': 'background: #FFF;'}
table_font = {'selector': 'tr', 'props': 'font-size: 10px;'}
table_data = {'selector': 'td', 'props': 'padding: 6px;'}
table_caption = {'selector': 'caption', 'props': 'font-size: 14px; font-style: italic; caption-side: bottom; text-align: left; margin-top: 10px'}
table_large_data = {'selector': 'tr', 'props': 'font-size: 14px;'}


table_css_styles = [even_rows, odd_rows, table_font, header_row, table_caption]
table_large_font = [even_rows, odd_rows, table_large_data, header_row, table_caption]

# this picks up the correct display names for the different languages
display_names = open_json("french_display_names.json")

land_use_names = open_json("french_land_use_names.json")

material_names = open_json("french_material_names.json")

group_names = open_json("french_group_names.json")

french_names = open_json("french_code_translations.json")

# Les variables aléatoires

Il s'agit de l'ensemble de données de base que vous pouvez obtenir une fois que vous avez supprimé les colonnes indésirables et appliqué un regroupement catégoriel. Ici, nous utilisons les groupes limités par la géographie, le temps, l'objet et l'ampleur.

## Le contexte de l'observation

Une fois les données collectées et agrégées, d'autres informations sont nécessaires pour les placer dans leur contexte par rapport aux autres observations.

Dans ce modèle, les étiquettes sont largement utilisées. Les objets, les lieux et les attributs des objets et des lieux ont tous des étiquettes. Les étiquettes relatives à l'utilisation des sols correspondent au rapport entre l'attribut et l'espace disponible.



```{attention}

_À l'origine, cette méthode était destinée à la surveillance des déchets sur le littoral. L'application de cette méthode aux rivières et aux sentiers de montagne fait l'objet de la présente discussion._

Avoir un consensus sur les descriptions d'objets et être discipliné dans les données enregistrées : c'est la meilleure façon de capturer les informations de l'observateur.

* Une fois le consensus atteint, les ingénieurs peuvent se mettre au travail.

* Il est plus facile de chercher quelque chose si vous savez ce que vous cherchez ou si vous savez ce qui est important.
```

In [2]:
# start variables
# this is the observations from the begining of monitoring
# to the end of the federal report from 2021. 
start_date, end_date = "2015-11-15", "2022-12-31"
lake = 'lac-leman'
catchment = 'rhone'
some_quants = [.03, .25, .48, .5, .52, .75, .97]
cantons = ["Vaud", "Valais", "Genève"]
columns = ['loc_date', 'slug', 'city', 'water_name_slug', 'date', 'doy', 'Project',
       'code', 'pcs/m', 'quantity', 'canton', 'zone',]

wt_data = pd.read_csv("resources/survey_work_data.csv", low_memory=False)

# test_data = wt_data[wt_data.Project.isin(["Testing", "testing"])].copy()
# test_data["Project"] = "testing"

# train_data = wt_data[wt_data.Project.isin(["Training", "training"])].copy()
# train_data["Project"] = "training"

# wt_data = pd.concat([test_data, train_data])
# wt_data.drop("location", axis=1, inplace=True)
# wt_data.to_csv("resources/survey_work_data.csv")

In [3]:


wt_data = wt_data[columns][wt_data.canton.isin(cantons)].copy()


# codes and beaches
dfCodes = pd.read_csv("resources/codes.csv")
dfBeaches = pd.read_csv("resources/beaches.csv")
dfBeaches.set_index("slug", inplace=True)
dfBeaches = dfBeaches[dfBeaches.water_name_slug == lake].copy()

### Résultats d'inventaire

```{important}
C'est ce qui est compté. La manière dont ces objets sont classés est un élément essentiel des données. C'est aussi la façon la plus simple de faire mesurer une valeur. Il est plus facile et plus rapide de le faire au moment de l'enquête.
```
#### Début: nov 2015, fin: décembre 2022

In [4]:
caption = "L'inventaire de base, jdl = jour de l'année. Échantillons est l'identifiant de \
l'inventaire, il connecte les objets au même inventaire. La quantité \
et les pcs/m sont les variables numériques pour chaque ligne. Le \
reste des colonnes sont des étiquettes."

combined_d = wt_data.rename(columns=display_names)

combined_d[combined_d.zone == "lac"].head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3)

,échantillons,site,municipalités,zone d'intérêt,date,jdl,Projet,code,pcs/m,quantité,canton,zone
0,"('aubonne', '2022-02-06')",aubonne,Aubonne,lac-leman,2022-02-06,37.000,testing,G10,0.000,0.000,Vaud,lac
1,"('aubonne', '2022-02-06')",aubonne,Aubonne,lac-leman,2022-02-06,37.000,testing,G100,0.000,0.000,Vaud,lac
2,"('aubonne', '2022-02-06')",aubonne,Aubonne,lac-leman,2022-02-06,37.000,testing,G101,0.000,0.000,Vaud,lac
3,"('aubonne', '2022-02-06')",aubonne,Aubonne,lac-leman,2022-02-06,37.000,testing,G103,0.020,1.000,Vaud,lac
4,"('aubonne', '2022-02-06')",aubonne,Aubonne,lac-leman,2022-02-06,37.000,testing,G106,0.100,5.000,Vaud,lac


### Détails du site d'inventaire

Celui-ci contient les détails de localisation tels que le GPS, le bassin fluvial de la ville, etc. Une grande partie a déjà été ajoutée aux données pour le travail. Cependant, les noms propres peuvent être extraits et des attributs peuvent être ajoutés à cet ensemble, puis appliqués.

La localisation GPS constitue le centre d'un hexagone de 3 000 mètres. La configuration de l'utilisation du sol pour chaque emplacement est configurée à partir de ce point.

```{note}
Les données utilisées pour identifier un emplacement sont définies sur la manière dont les informations vont être utilisées. C’est également un moyen simple d’améliorer le flux de données une fois les exigences comprises.
```

In [5]:
beach_columns = ['location', 'latitude', 'longitude',
       'water_name', 'city',
       'river_bassin']

caption = "Détails de localisation des plages du Léman. Des attributs \
peuvent être ajoutés à cet ensemble de données et transmis ultérieurement \
si nécessaire. Pour l'instant, les caractéristiques topographiques sont \
stockées sur un autre ensemble de données partageant le même index."
dfBeaches.index.name = None

    
dfBeaches[beach_columns].rename(columns=display_names).head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3)

,site,latitude,longitude,zone d'interêt,municipalités,bassin versant
anarchy-beach,Anarchy Beach,46.447,6.860,Lac Léman,La Tour-de-Peilz,rhone
arabie,Arabie,46.462,6.835,Lac Léman,Vevey,rhone
baby-plage-geneva,Baby Plage Geneva,46.209,6.163,Lac Léman,Genève,rhone
baby-plage-ii-geneve,Baby-Plage II Genève,46.209,6.164,Lac Léman,Genève,rhone
bain-des-dames,Bain des Dames,46.451,6.858,Lac Léman,La Tour-de-Peilz,rhone


### Catégories d'objets

Les catégories d'objets correspondent à la manière dont la personne transmet l'observation à l'appareil. La description et le nom du groupe permettent aux administrations régionales d'obtenir davantage d'informations à partir des données.

```{note}
En créant des groupes d'objets qui représentent des cas d'utilisation, il peut être plus facile pour les acteurs régionaux d'identifier les priorités. Il s'agit peut-être de la meilleure voie de discussion entre les acteurs régionaux. Si les catégories d'objets ou les groupes correspondent à des priorités communes, les communications peuvent être simplifiées.
```

In [6]:
caption = "Tableau d'informations sur les objets. Nous pouvons ajouter des codes spécifiques à \
une région. Par exemple nous ajoutons le code G708 pour les bâtons de ski. \
Ils ne sont pas communs au bord des lacs mais on les trouve en montagne. \
Un autre exemple est celui des contenants d’appâts aux phéromones. \
Ils sont utilisés dans les vignobles et on les retrouve là où se concentrent les vignobles."

# translate code data to language of choice
dfCodes["material"] = dfCodes.material.map(lambda x: material_names[x])
dfCodes["description"] = dfCodes.code.apply(lambda x: french_names[x])
dfCodes["groupname"] = dfCodes.groupname.apply(lambda x: group_names[x])

dfCodes.set_index("code", inplace=True)
dfCodes.index.name = None

dfCodes.rename(columns=display_names).head().style.set_table_styles(table_large_font).set_caption(caption)

,matériel,description,code de base,groupe
G708,Métal,Batons de ski,G199,loisirs
G212,Produit chimiques,Nodules de petrol,Parent code,non définie
G213,Produit chimiques,Paraffine,Parent code,loisirs
G214,Produit chimiques,Goudron,Parent code,infrastructures
G135,Textiles,"Vêtements, chaussures, chapeaux et gants",Parent code,articles personnels


### Utilisation du sol

Il s’agit de l’ampleur des caractéristiques topographiques trouvées dans un rayon de 1 500 du lieu d’enquête. La source est la collection de cartes swissTLMRegio. Une superposition hexagonale est appliquée aux couches de couverture terrestre et d’utilisation des terres. Ensuite, le contenu de chaque hex est enregistré en mètres² ou en mètres linéaires.

Ce processus a montré des corrélations entre l'ampleur des résultats de l'enquête pour certains objets et l'ampleur de l'élément topographique. Ce processus a été expliqué dans [Near of Far](https://hammerdirt-analyst.github.io/landuse/probability.html) et  [Land use profile](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/land_use_correlation.html).

L'ampleur de l'élément topographique et sa valeur mise à l'échelle sont disponibles. Les données ont été séparées en groupes percentiles. Il existe 21 groupes, un groupe étant destiné au cas où la caractéristique d'utilisation des terres n'est pas présente, puis un groupe pour chaque cinquième percentile.

```{note}

Pour cette phase initiale, seules les couches cartographiques de base ont été utilisées. Pour les partenaires intéressés, l’une des couches cartographiques de[TLMRegio](https://www.swisstopo.admin.ch/de/geodata/landscape/tlmregio.html) fonctionnera sur ces données. Cela permet aux équipes indépendantes d’utiliser plus facilement les résultats pour des besoins spécifiques.

```

In [7]:
land_use = pd.read_csv("resources/work_land_use.csv")

caption = "Le profil d'occupation du sol de la tiger-duck-beach à \
Saint Sulpice. Rappelez-vous que la grandeur des rues est en mètres, \
tout le reste est en mètres carrés. La quantité de terre sèche est \
donnée en mètres², normalisée étant le résultat de la magnitude/terre sèche. \
Rang est le classement de cette caractéristique d'utilisation des sols (0 - 20)."

index_order = [
    'rues',
    'bâtiments',
    'centre ville',
    'forêt', 
    'vergers',
    'vignobles',
    'infrastructure', 
    'récréation',
    'non définie',  
   
]


# display land use for one location
lud = land_use[land_use.slug == "grand-clos"].rename(columns=display_names)
lud = lud[~lud.utilisation.isin(["See", "dist", "length"])].copy()
lud["rang"] = lud.rang.astype(int)
lud["magnitude"] = lud.magnitude.astype(int)
lud["utilisation"] = lud["utilisation"].apply(lambda x: land_use_names[x])
lud["normalisée"] = lud["normalisée"].round(3)
lud.set_index("utilisation", inplace=True)
lud = lud.reindex(index_order)
lud.drop("site", axis=1, inplace=True)
lud.index.name = None
lud.style.set_table_styles(table_large_font).set_caption(caption).format('{:,}')

,normalisée,terre sèche,magnitude,rang
rues,0.142,0,"35,120",3
bâtiments,0.272,"2,866,568","781,085",6
centre ville,0.0,0,0,0
forêt,0.48,"2,866,568","1,377,375",10
vergers,0.0,0,0,0
vignobles,0.0,0,0,0
infrastructure,0.015,"2,866,568","42,872",1
récréation,0.033,"2,866,568","95,027",1
non définie,0.247,"2,866,568","708,108",5


### Cantonal

La situation géographique ciblée. Cependant, les échantillons du Rhône et du Léman partagent des origines communes. Pour le Léman les échantillons du Valais représentent 5% de tous les échantillons et 12% en quantité.

```{note}
Cela devrait affecter les décisions d’échantillonnage au niveau régional. Pour le Valais le bilan n'est pas correct compte tenu des kilomètres du Rhône.

Il existe cependant un consensus entre 2021 et 2022 en référence aux valeurs élevées à l'embouchure du Rhône.
```

In [8]:
# the values to be compared
operation = {"quantity":"sum", "city":"nunique", "loc_date":"nunique", "slug":"nunique"}

# lac leman only:
combined = wt_data[(wt_data.canton.isin(cantons))&(wt_data.zone == "lac")].copy()

# the totals for all the data
totals = combined.agg(operation)

# the totals for each canton
canton = combined.groupby("canton").agg(operation)

# the contribution of each canton to the total
canton["q_weight"] = (canton.quantity/totals.quantity).round(3)

# the number of samples contributed to the project by canton
canton["s_weight"] = (canton.loc_date/totals.loc_date).round(3)

# sum accross the bottom
canton.loc["Total"] = canton.sum()
canton[list(operation.keys())] = canton[operation.keys()].astype("int")
canton.index.name = None

caption = "Nous comptons 96 320 objets pour la période indiquée. \
De 25 villes. Il y avait 319 échantillons provenant de 56 sites."

canton.rename(columns=display_names).style.set_table_styles(table_large_font).format('{:,}').set_caption(caption)

,quantité,municipalités,échantillons,site,% de totale,% des échantillons
Genève,"12,481",4,50,11,0.13,0.157
Valais,"12,193",2,17,2,0.127,0.053
Vaud,"71,646",19,252,43,0.744,0.79
Total,"96,320",25,319,56,1.001,1.0


### Régional

Ces étiquettes sont une façon de diviser le lac. Le transport et le dépôt d'objets dans ces zones peuvent avoir une importance régionale. Si tel est le cas, le Valais est un contributeur important.

In [9]:
# the way the data is separated is analysis dependent
# this splits the lake up into three regions
regions = pd.read_csv("resources/lac_leman_regions.csv")
regions.drop_duplicates(inplace=True)
rg_map = regions.set_index("slug")

combined["region"] = combined.slug.apply(lambda x: rg_map.loc[x, "alabel"])
combined["zone"] = "lac"

# group by region 
regional = combined.groupby("region").agg(operation)

# get the weight of each region to the total
regional["q_weight"] = (regional.quantity/totals.quantity).round(3)
regional["s_weight"] = (regional.loc_date/totals.loc_date).round(3)
regional.loc["Total"] = regional.sum()

# display results
regional[list(operation.keys())] = regional[operation.keys()].astype("int")
regional.index.name = None

caption_regional_fr = "Les noms régionaux représentent différentes caractéristiques \
hydrologiques ainsi que des différences géographiques."

regional.rename(columns=display_names).style.format(precision=3).set_table_styles(table_large_font).format('{:,}').set_caption(caption_regional_fr)

,quantité,municipalités,échantillons,site,% de totale,% des échantillons
Grand lac,"24,385",10,83,21,0.253,0.26
Haut lac,"57,743",9,157,20,0.599,0.492
Petit lac,"14,192",6,79,15,0.147,0.248
Total,"96,320",25,319,56,0.999,1.0


### Municipal

L'unité de base. Les communes non inscrites sur la liste n'ont pas été échantillonnées.

In [10]:
municipal = combined.groupby("city").agg({"quantity":"sum", "loc_date":"nunique", "slug":"nunique"})
municipal["q_weight"] = (municipal.quantity/totals.quantity).round(3)
municipal["s_weight"] = (municipal.loc_date/totals.loc_date).round(3)
municipal.loc["Total"] = municipal.sum().round(2)
municipal[["quantity", "loc_date", "slug"]] = municipal[["quantity", "loc_date", "slug"]].astype("int")
municipal.index.name = None
caption_municipal_fr = "Montreux et Vevey représentent 31% des échantillons depuis 2015."
municipal.rename(columns=display_names).style.set_table_styles(table_large_font).format('{:,}').set_caption(caption_municipal_fr)

,quantité,échantillons,site,% de totale,% des échantillons
Allaman,631,3,1,0.007,0.009
Aubonne,193,4,1,0.002,0.013
Bourg-en-Lavaux,121,2,1,0.001,0.006
Bouveret,"3,286",4,1,0.034,0.013
Clarens,"1,128",4,1,0.012,0.013
Crans,21,3,1,0.0,0.009
Cully,48,4,1,0.0,0.013
Genève,"8,129",33,6,0.084,0.103
Gland,"1,690",26,3,0.018,0.082
Grangettes,"3,677",4,1,0.038,0.013


### Échantillons de bord des rivières


Les rivières représentent un environnement d'échantillonnage différent de celui des lacs. La marque des hautes eaux change plus fréquemment et avec plus d'ampleur sur les rivières. Ces changements sont si importants que les relevés effectués sur les berges impliquent souvent de retirer les sacs des arbres.

```{attention}

Les acteurs régionaux doivent se mettre d'accord sur les lieux d'échantillonnage optimaux le long de la voie navigable. Cela implique l'apport d'hydrologues et d'autres disciplines qui pourraient indiquer des lieux d'accumulation.

Il s'agit notamment d'examiner attentivement les limites de la zone d'enquête et la manière de définir ces limites sur le terrain.
``` 

In [11]:
display_columns = [
    'loc_date', 'slug', 'city', 'water_name_slug', 'date', 'doy', 'Project', 'code', 'pcs/m', 'quantity', 'canton','zone'
]

caption_rivers_fr = "Les échantillons de rivière ont la même structure que \
les échantillons de lac. La plupart des échantillons de rivière ont été prélevés en 2018."

wt_data[wt_data.zone == "rivière"][display_columns[1:]].rename(columns=display_names).head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3).set_caption(caption_rivers_fr)

,site,municipalités,zone d'intérêt,date,jdl,Projet,code,pcs/m,quantité,canton,zone
10387,arve_carouge_battistellak,Veyrier,arve,2017-04-17,107.000,training,Gcaps,0.080,4.000,Genève,rivière
10388,arve_carouge_battistellak,Veyrier,arve,2017-05-25,145.000,training,Gcaps,0.090,5.000,Genève,rivière
10389,arve_carouge_battistellak,Veyrier,arve,2017-06-27,178.000,training,Gcaps,0.050,3.000,Genève,rivière
10390,arve_carouge_battistellak,Veyrier,arve,2017-07-31,212.000,training,Gcaps,0.070,4.000,Genève,rivière
10391,arve_carouge_battistellak,Veyrier,arve,2017-08-28,240.000,training,Gcaps,0.110,6.000,Genève,rivière


### Échantillons sentiers pédèstre

Le lien entre les résultats de ces sites et le lac n'est pas clair. Nous ne disposons pas de données antérieures à 2020 pour les Alpes et les associations n'ont pas manifesté d'intérêt soutenu pour continuer ce projet.

```{important}
Ces résultats sont intéressants car ils représentent des zones où l'on ne s'attend pas à trouver des déchets. Dans ce contexte, les échantillons prélevés sur les sentiers de randonnée sont équivalents à ceux prélevés sur les plages. Des efforts considérables sont également déployés à tous les niveaux pour entretenir ces ressources.

L'élaboration d'un calendrier d'échantillonnage et la définition d'objectifs régionaux à cet égard peuvent inciter d'autres acteurs à agir.
```

In [12]:
alpes_locs =  {
    'airolo': "Ticino" ,
    'andermatt': "Uri" ,
    'cabanes-des-diablerets': "Vaud",
    'charmey': "Valais",
    'crans-montana': "Valais",
    'grindelwald': "Bern",
    'la-berra': "Fribourg",
    'la-robella':"Neuchâtel",
    'la-tzoumaz':  "Valais",
    'les-crosets': "Valais",
    'les-diablerets': "Vaud",
    'les-paccots': "Fribourg",
    'morgins':"Valais",
    'nendaz': "Valais",
    'san-bernardino': "Grisons" ,
    'val-calanca': "Ticino",
    'verbier':"Valais",
    'veysonnaz': "Valais",
    'villars': "Vaud",
    'monte-generoso': "Ticino"
}



caption_trails_fr = "Les données relatives aux sentiers de randonnée proviennent du rapport fédéral. \
Elles ont été réalisées par deux personnes au cours d'une saison estivale."

wt_data[wt_data.zone == "montagne"][display_columns[1:]].rename(columns=display_names).head().style.set_table_styles(table_large_font).set_caption(caption).format(precision=3).set_caption(caption_trails_fr)

,site,municipalités,zone d'intérêt,date,jdl,Projet,code,pcs/m,quantité,canton,zone
6057,cabanes-des-diablerets,Ormont-Dessus,alpes-vaudoises,2021-08-28,240.000,testing,Gcaps,0.000,0.000,Vaud,montagne
6058,charmey,Val-de-Charmey,prealpes-fribourgeoises,2021-05-08,128.000,testing,Gcaps,0.100,3.000,Valais,montagne
6059,crans-montana,Crans-Montana,alpes-valaisannes,2021-06-12,163.000,testing,Gcaps,0.000,0.000,Valais,montagne
6063,la-tzoumaz,Riddes,alpes-valaisannes,2021-05-22,142.000,testing,Gcaps,0.000,0.000,Valais,montagne
6064,les-crosets,Val-d'Illiez,alpes-valaisannes,2021-06-06,157.000,testing,Gcaps,0.000,0.000,Valais,montagne


### Données dimensionnelles

Nous ne listons pas les poids ici. Les méthodes utilisées pour enregistrer les poids diffèrent d'un groupe à l'autre. Cependant nous disposons de la longueur du rivage ou du sentier, de la superficie et du temps en minutes.

```{note}
Le temps nécessaire à la réalisation d'un inventaire est souvent sous-estimé. Des inventaires de plusieurs milliers d'unités peuvent nécessiter 8 à 12 heures de comptage, mais seulement une heure de collecte.

En fin de compte, la précision de vos données dépend des ressources allouées à cet effet. D'après notre expérience, les gens participent à la collecte mais pas au comptage.
```

In [13]:
working_dimensions = pd.read_csv('resources/work_dimensions.csv')
working_dimensions.set_index("slug", inplace=True)
working_dimensions.index.name = None

caption_dims_fr = "l'ampleur de l'effort en termes de surface, de distance et de temps."
working_dimensions.rename(columns=display_names).head().style.set_table_styles(table_large_font).format(precision=3).set_caption(caption_dims_fr)

,m²,mètres,minutes
airolo,57.000,38,99
andermatt,800.000,100,220
cabanes-des-diablerets,18.000,12,199
charmey,150.000,30,220
crans-montana,64.000,43,149


### Test

Avant de passer à l'étape suivante. Nous vérifions les données par rapport au rapport fédéral. Nos résultats ici doivent correspondre à la page ici [Le Léman 2021](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/lac-leman.html)

In [14]:
# check the 2021 results against the pubished federal report
start_date, end_date = pd.to_datetime("2020-04-01"), "2021-05-31" 
combined["date"] = pd.to_datetime(combined["date"])
mask = (combined["date"] >= start_date) & (combined["date"] <= end_date) & (combined.zone == 'lac')
iqaasl = combined[mask]
codes = ["Gfrags", "G27", "G67", "G95", "G112", "G30", "G70", "G89"]

t_c_v = iqaasl[iqaasl.code.isin(codes)]
tcv = t_c_v.groupby(["loc_date", "code"], as_index=False).agg({"pcs/m":"sum", "quantity":"sum"})
tfg = tcv.groupby("code", as_index=False)["pcs/m"].median().T
tfg.columns = tfg.loc["code"].values
tfg.drop("code", inplace=True)
caption_test_fr = "Ces valeurs doivent correspondre à la médiane déclarée pour le même objet dans le rapport fédéral. "
tfg.style.set_table_styles(table_large_font).format(precision=3).set_caption(caption_test_fr)

,G112,G27,G30,G67,G70,G89,G95,Gfrags
pcs/m,0.000,0.470,0.215,0.090,0.010,0.055,0.120,0.615


## L'ensemble de données pour le lac

Les résultats de l'enquête contiennent désormais toutes les informations décrites ici ou permettent d'accéder à d'autres informations sur l'événement.

In [15]:
combined["date"] = combined["date"].dt.date

caption_last_fr = "Résultats pour les emballages de bonbons. Les données de l'enquête comportent \
des étiquettes pour les relations inter et intra cantonales. \
C'est essentiel pour les replacer dans leur contexte."

combined[combined.code == "G30"].rename(columns=display_names).head().style.set_table_styles(table_css_styles).format(precision=3).set_caption(caption_last_fr)

,échantillons,site,municipalités,zone d'intérêt,date,jdl,Projet,code,pcs/m,quantité,canton,zone,region
36,"('aubonne', '2022-02-06')",aubonne,Aubonne,lac-leman,2022-02-06,37.000,testing,G30,0.100,5.000,Vaud,lac,Grand lac
110,"('aubonne', '2022-05-14')",aubonne,Aubonne,lac-leman,2022-05-14,134.000,testing,G30,0.220,11.000,Vaud,lac,Grand lac
184,"('aubonne', '2022-09-10')",aubonne,Aubonne,lac-leman,2022-09-10,253.000,testing,G30,0.060,3.000,Vaud,lac,Grand lac
258,"('aubonne', '2022-11-05')",aubonne,Aubonne,lac-leman,2022-11-05,309.000,testing,G30,0.060,3.000,Vaud,lac,Grand lac
332,"('baby-plage', '2022-01-22')",baby-plage,Genève,lac-leman,2022-01-22,22.000,testing,G30,0.111,9.000,Genève,lac,Petit lac


In [16]:

today = dt.datetime.now().date().strftime("%d/%m/%Y")
where = "Biel, CH"

my_block = f"""
<br />
<br />
This script updated {today} in {where}

\u2764\ufe0f what you do - faites ce que vous \u2764\ufe0f
"""

md(my_block)


<br />
<br />
This script updated 10/09/2023 in Biel, CH

❤️ what you do - faites ce que vous ❤️


In [17]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/cantonal_reports.git

Git branch: main

pandas : 2.0.3
numpy  : 1.25.2
json   : 2.0.9
slugify: 8.0.1



_analyst at hammerdirt_